# Initialization

In [21]:
import math

import numpy as np
import pandas as pd
import darkchem

# Encoding Rxns

## Alkylation of Enolates

1. $CH_{3}C(O)CH_{3} + OH^{-} + CH_{3}Br \rightarrow CH_{3}C(O)CH_{2}CH_{3} + H_{2}O + Br^{-}$
2. Cyclohexanone

In [2]:
smiles = {'reactants': ['CC(=O)C', 'O=C1CCCCC1', 'O=C1CCCC1C', 'O=C1CCCC1C'], 
          'products': ['CCC(=O)C', 'CC1CCCCC1=O', 'CC1CCC(C1=O)C', 'O=C1CCCC1(C)C']}

In [7]:
smiles['rvec'] = [darkchem.utils.struct2vec(reactant).astype(np.int16) for reactant in smiles['reactants']]
smiles['pvec'] = [darkchem.utils.struct2vec(product).astype(np.int16) for product in smiles['products']]
smiles['rxnvector'] = [smilesdf.pvec[i] - smilesdf.rvec[i] for i in range(len(smilesdf))]

In [15]:
smilesdf = pd.DataFrame(smiles)
smilesdf.head()

,reactants,products,rvec,pvec,rxnvector
0,CC(=O)C,CCC(=O)C,"[13, 13, 21, 29, 16, 22, 13, 0, 0, 0, 0, 0, 0,...","[13, 13, 13, 21, 29, 16, 22, 13, 0, 0, 0, 0, 0...","[0, 0, -8, -8, 13, -6, 9, 13, 0, 0, 0, 0, 0, 0..."
1,O=C1CCCCC1,CC1CCCCC1=O,"[16, 29, 13, 1, 13, 13, 13, 13, 13, 1, 0, 0, 0...","[13, 13, 1, 13, 13, 13, 13, 13, 1, 29, 16, 0, ...","[-3, -16, -12, 12, 0, 0, 0, 0, -12, 28, 16, 0,..."
2,O=C1CCCC1C,CC1CCC(C1=O)C,"[16, 29, 13, 1, 13, 13, 13, 13, 1, 13, 0, 0, 0...","[13, 13, 1, 13, 13, 13, 21, 13, 1, 29, 16, 22,...","[-3, -16, -12, 12, 0, 0, 8, 0, 0, 16, 16, 22, ..."
3,O=C1CCCC1C,O=C1CCCC1(C)C,"[16, 29, 13, 1, 13, 13, 13, 13, 1, 13, 0, 0, 0...","[16, 29, 13, 1, 13, 13, 13, 13, 1, 21, 13, 22,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 13, 22, 13, 0, ..."


In [33]:
np.corrcoef(smiles['rxnvector'])

array([[ 1.        , -0.00168265,  0.06415584, -0.01032847],
       [-0.00168265,  1.        ,  0.71701091,  0.34875671],
       [ 0.06415584,  0.71701091,  1.        ,  0.78291073],
       [-0.01032847,  0.34875671,  0.78291073,  1.        ]])